In [2]:
# Imports necessary modules
from suds.client import Client
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import datetime
import time

# Imports matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

# Data trimming function
def acerta_data(data):
    try: 
        a = datetime.datetime.strptime(data, '%d/%m/%Y').strftime('%m/%d/%Y')
    except:    
        pass
    try:
        a = datetime.datetime.strptime(data, '%m/%Y')
        if a.month == 12:
            a = datetime.date(a.year, a.month, 31).strftime('%m/%d/%Y')
        else:
            a = datetime.date(a.year, a.month+1, a.day) - datetime.timedelta(days=1)
            a = a.strftime('%m/%d/%Y')
    except:
        pass
    try:    
        a = datetime.date(int(data), 12, 31).strftime('%m/%d/%Y')
    except:
        pass
    return a

# Update select attributes of the list
def atualizar_atributos(novoscods):
#   novoscods = pd.read_csv('./novoscodigos_bcb_2.csv')
#   novoscods.insert(len(novoscods.columns),'per', '')
#   novoscods.insert(len(novoscods.columns),'anoinicio', '')
#   novoscods.insert(len(novoscods.columns),'unidade', '')
    for x in range(len(novoscods.index)):
        try:
            jsnn = str(bcb.service.getUltimoValorVO(novoscods['cod'][x]))
            jsnn1 = jsnn[11:]
            jsnn2 = jsnn1.replace('\n', '=')
            jsnn2 = jsnn2.split('=')
            jsnn3 = dict(zip(jsnn2[1::2], jsnn2[2::2]))
            jsnn3 = {k.strip():v.strip() for k, v in jsnn3.items()}
            novoscods.set_value(x, 'per', jsnn3['periodicidadeSigla'])
            novoscods.set_value(x, 'anoinicio', jsnn3['anoInicio'])
            novoscods.set_value(x, 'unidade', jsnn3['unidadePadrao'])
            print(x)
        except Exception:
            continue
    return

# Refresh code list
def atualizar_codigos():
    for x in range(25000):
        try:
            xml2 = bcb.service.getUltimoValorXML(x)
            root2 = ET.fromstring(xml2)
            for n in root2.findall('SERIE'):
                codat = pd.Series({'cod': n.find('CODIGO').text, 'descr': n.find('NOME').text})
                novoscods = novoscods.append(codat, ignore_index=True)
            print(x)
        except Exception:
            continue
    return novoscods

            
# Loads series indexes for lookup
series_bcb = pd.read_csv('./novoscodigos_bcb_per_idx.csv', index_col=0, header=0)
series_bcb = series_bcb.drop_duplicates()
# Lookup expression
#   series_bcb[series_bcb['Descrição'].str.contains("Dólar")==True]

print('Acesso à API do Banco Central do Brasil (v. 0.2)')

# User input
series_selecionadas = []
# String searching on the database
while True:
    search_string = input("\nDigite parte do nome da série que deseja (p.ex. 'dólar')\nou FIM para prosseguir: ")
    if search_string == 'FIM':
        break
    else:
        result = series_bcb[series_bcb['descr'].str.contains(search_string, case=False)==True]
        result = result.sort_values('descr')
        result.insert(0,'Cód',result.index)
        result.index = range(1, len(result.index)+1)
        for n in range(1, len(result.index)+1):
            print('[',str(n).rjust(5),']', result['descr'][n], result['per'][n], result['anoinicio'][n], result['unidade'][n])
        # User selection
        series_input = []
        series_input.extend(list(int(x) for x in input("\nDigite o número das séries que deseja (separe com um espaço): ").split()))
        # Transforming user input in usable indexes
        for n in range(len(series_input)):
            series_input[n] = result.get_value(series_input[n],'Cód')
        series_selecionadas.extend(series_input)

series_selecionadas_nom = {}
for x in series_selecionadas:
    series_selecionadas_nom[x] = series_bcb.get_value(x, 'descr')
    
datainicio = str(input("\nData inicial (DD/MM/AAAA): ") or '01/01/2000')
datafim = str(input("Data final (DD/MM/AAAA): ") or '01/09/2016')
nome_output = input("Nome do arquivo de saída: ") or 'BCB_SGS'

# Creates Pandas DataFrame with calandar days as index
index = pd.date_range(datetime.datetime.strptime(datainicio, '%d/%m/%Y').strftime('%m/%d/%Y'), datetime.datetime.strptime(datafim, '%d/%m/%Y').strftime('%m/%d/%Y'))
df = pd.DataFrame(index=index, columns=series_selecionadas)

# Activates WSDL and retrieves time series
url = 'https://www3.bcb.gov.br/sgspub/JSP/sgsgeral/FachadaWSSGS.wsdl'
bcb = Client(url)
try:
    for n in range(len(series_selecionadas)):
        print('\nBuscando série ', repr(series_selecionadas_nom[series_selecionadas[n]]),'.')
        dados = bcb.service.getValoresSeriesXML([series_selecionadas[n]], datainicio, datafim)
        # Parse XML data
        root = ET.fromstring(dados)
        # Updates the DataFrame values
        for serie in range(len(root)):
            for ponto in range(len(root[serie])):
                df.set_value(acerta_data(root[serie][ponto][0].text), int(root[serie].attrib['ID']), float(root[serie][ponto][1].text))
        df = df.apply(lambda x: pd.to_numeric(x, errors='ignore'))
        print('Aguardando 2 segundos...')
        time.sleep(2)

except Exception as detail:
    print(detail)

# Previous list generator
#   datas = []
#   for serie in range(len(root)):
#       for ponto in range(len(root[serie])):
#              datas.append([int(root[serie].attrib['ID']), root[serie][ponto][0].text, float(root[serie][ponto][1].text)])
# Equivalent comprehension
#   [(int(root[serie].attrib['ID']), root[serie][ponto][0].text, float(root[serie][ponto][1].text)) for serie in range(len(root)) for ponto in range(len(root[serie]))]

# Export to CSV file
df2 = df.dropna()
df2 = df2.rename(columns=series_selecionadas_nom)
df2.to_csv('~/Desktop/'+nome_output+'.csv')
print('\nArquivo {arquivo} gravado com sucesso.'.format(arquivo=nome_output))

Acesso à API do Banco Central do Brasil (v. 0.2)

Digite parte do nome da série que deseja (p.ex. 'dólar')
ou FIM para prosseguir: cdi
[     1 ] Taxa de juros - CDI "D" 1986.0 "% a.d."
[     2 ] Taxa de juros - CDI acumulada no mês "M" 1986.0 "% a.m."
[     3 ] Taxa de juros - CDI acumulada no mês anualizada base 252 "M" 1986.0 "% a.a."
[     4 ] Taxa de juros - CDI anualizada base 252 "D" 1986.0 "% a.a."

Digite o número das séries que deseja (separe com um espaço): 2

Digite parte do nome da série que deseja (p.ex. 'dólar')
ou FIM para prosseguir: ibovespa
[     1 ] Ibovespa - Variação percentual mensal "M" 1987.0 "%"
[     2 ] Valor das empresas listadas no Ibovespa "M" 1996.0 "u.m.c. (milhões)"

Digite o número das séries que deseja (separe com um espaço): 1

Digite parte do nome da série que deseja (p.ex. 'dólar')
ou FIM para prosseguir: desocup
[     1 ] Desocupados - Total "M" 2001.0 "Unidades (mil)"
[     2 ] Desocupados - Total - Desativada "M" 1994.0 "Unidades"
[     3 ] Dist

In [5]:
df2

,Taxa de juros - CDI acumulada no mês,Ibovespa - Variação percentual mensal,Taxa de desocupação - PNADC
2012-03-31,0.81,-1.98,7.9
2012-04-30,0.70,-4.17,7.8
2012-05-31,0.73,-11.86,7.6
2012-06-30,0.64,-0.25,7.5
2012-07-31,0.68,3.21,7.4
2012-08-31,0.69,1.72,7.3
2012-09-30,0.54,3.70,7.1
2012-10-31,0.61,-3.56,6.9
2012-11-30,0.54,0.71,6.8
2012-12-31,0.53,6.05,6.9


In [7]:
df2.describe()

,Taxa de juros - CDI acumulada no mês,Ibovespa - Variação percentual mensal,Taxa de desocupação - PNADC
count,52.000000,52.000000,52.000000
mean,0.838846,-0.287885,7.788462
std,0.200828,6.095485,1.299269
min,0.480000,-11.860000,6.200000
25%,0.697500,-3.990000,6.900000
50%,0.820000,-0.765000,7.400000
75%,1.010000,3.685000,8.150000
max,1.180000,16.970000,11.300000
